In [1]:
%matplotlib notebook
%matplotlib inline
# Dependencies
from census import Census
from config import (census_key, gkey)
import gmaps
import numpy as np
import pandas as pd
import requests
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt
import json
from pprint import pprint
import os
import random
from itertools import accumulate
import geopandas as gpd #pip install geopandas or conda install -c conda-forge geopandas
from shapely.geometry import Point, Polygon #pip install shapely
from descartes import PolygonPatch #pip install descartes
import gmaps.geojson_geometries
from matplotlib.cm import viridis
from matplotlib.colors import to_hex
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

# Census API Key
c = Census(census_key, year=2017)

In [2]:
usa = gpd.read_file("states_21basic")
usa.head()

,drawseq,state_abbr,state_fips,state_name,sub_region,geometry
0,NaN,HI,15,Hawaii,Pacific,"MULTIPOLYGON (((-160.07380 22.00418, -160.0497..."
1,NaN,WA,53,Washington,Pacific,"MULTIPOLYGON (((-122.40202 48.22522, -122.4628..."
2,NaN,MT,30,Montana,Mountain,"POLYGON ((-111.47543 44.70216, -111.48080 44.6..."
3,NaN,ME,23,Maine,New England,"MULTIPOLYGON (((-69.77728 44.07415, -69.85993 ..."
4,NaN,ND,38,North Dakota,West North Central,"POLYGON ((-98.73044 45.93827, -99.00683 45.939..."


In [3]:
# usa.plot()

In [4]:
gmaps.configure(api_key=gkey)

counties_geojson = gmaps.geojson_geometries.load_geometry('us-counties')

fig = gmaps.Map()

gini_layer = gmaps.geojson_layer(counties_geojson)
fig.add_layer(gini_layer)
# fig

In [5]:

import csv

with open('income.csv', newline='') as f:
    reader = csv.reader(f)
    res = list(map(tuple, reader))

incomes= dict(res)

# swapping FIPS and income so I can edit FIPS to have leading zeros
incomess = {}
for key, value in incomes.items():
    incomess[value] = key
# incomess


In [6]:
# adding leading zeros to fips
for k, v in incomess.items():
    incomess[k] = str(v).zfill(3)
# incomess

In [7]:
# swapping back
incomes = {}
for key, value in incomess.items():
    incomes[value] = key
inc = {}
for k, v in incomes.items():
    inc[k] = float(v)
# inc

In [8]:
# counties_geojson['features']

In [9]:
import matplotlib.pyplot as plt
import matplotlib as mpl

# convert values in dict to float
values = [float(x) for x in list(incomes.values())]
# We will need to scale the income values to lie between 0 and 1
min_income = min(inc.values())
max_income = max(inc.values())
income_range = (max_income - min_income)

def calculate_color(inc):

    # make income a number between 0 and 1
    normalized_income = (inc - min_income) / income_range

    # invert income so that high inequality gives dark color
    inverse_income = 1.0 - normalized_income

    # transform the incomes coefficient to a matplotlib color
    mpl_color = viridis(inverse_income)

    # transform from a matplotlib color to a valid CSS color
    gmaps_color = to_hex(mpl_color, keep_alpha=False)
#     gmaps_color= gmaps_color
    
    return gmaps_color


In [10]:
colors = []
for feature in counties_geojson['features']:
    county_FIPS = feature['properties']['COUNTY']
    try:
        incom = incomes[county_FIPS]
        color = calculate_color(inc)
    except KeyError:
        # no income for that county: return default color
        color = (0, 0, 0, 0.3)
    colors.append(color)
    
colors

TypeError: unsupported operand type(s) for -: 'dict' and 'float'

In [ ]:
fig = gmaps.Map()
income_layer = gmaps.geojson_layer(counties_geojson, fill_color=colors, stroke_color=colors, fill_opacity=0.8)
fig.add_layer(income_layer)
fig

In [ ]:
education_percentages = pd.read_csv("education_percentages.csv")
education_percentages.head()

In [ ]:
import gmaps.datasets
rows = gmaps.datasets.load_dataset('gini') # 'rows' is a list of tuples
country2gini = dict(rows) # dictionary mapping 'country' -> gini coefficient
print(country2gini['United Kingdom'])
rows

In [ ]:
# We will need to scale the GINI values to lie between 0 and 1
min_gini = min(country2gini.values())
max_gini = max(country2gini.values())
gini_range = max_gini - min_gini

def calculate_color(gini):
    """
    Convert the GINI coefficient to a color
    """
    # make gini a number between 0 and 1
    normalized_gini = (gini - min_gini) / gini_range

    # invert gini so that high inequality gives dark color
    inverse_gini = 1.0 - normalized_gini

    # transform the gini coefficient to a matplotlib color
    mpl_color = viridis(inverse_gini)

    # transform from a matplotlib color to a valid CSS color
    gmaps_color = to_hex(mpl_color, keep_alpha=False)

    return gmaps_color


In [ ]:
colors = []
for feature in countries_geojson['features']:
    country_name = feature['properties']['name']
    try:
        gini = country2gini[country_name]
        color = calculate_color(gini)
    except KeyError:
        # no GINI for that country: return default color
        color = (0, 0, 0, 0.3)
    colors.append(color)

In [ ]:
ethnicity_percentages = pd.read_csv("ethnicity_percentages.csv")
ethnicity_percentages.head()

In [ ]:
income = pd.read_csv("income_median_age.csv")
income = income[["State Name", "Household Income", "Median Age", "Median Male Age", "Median Female Age"]]
income.head()

In [ ]:
total_data = pd.read_csv("final_df.csv")
total_data.head()

In [ ]:
gmaps.configure(api_key=gkey)

In [ ]:
# state_coords = pd.DataFrame.from_dict(statecoords.items())
 
# state_coords[0][0]

In [ ]:
states = pd.read_csv("mycsvfile.csv")
for i in states:
    print(i, states[i][:])